In [19]:
import pandas as pd
from mesa import Agent, Model
from mesa.datacollection import DataCollector
from mesa.time import RandomActivation
from mesa.time import StagedActivation
from mesa.space import MultiGrid
import os

In [20]:
# Voter Agent
class Voter(Agent):
    def __init__(self, unique_id, model, attributes):
        super().__init__(unique_id, model)
        self.attributes = attributes

    def evaluate_candidate(self):
        min_difference = float('inf')
        chosen_candidate = None

        for candidate in self.model.candidates:
            # Calculate the weighted difference between voter and candidate attributes
            difference = sum(
                self.attributes[attr] * abs(self.attributes[attr] - getattr(candidate, attr))
                for attr, weight_attr in zip(
                    [
                        "government-intervention",
                        "government-control-of-economy",
                        "free-trade",
                        "protectionism",
                        "pro-choice",
                        "tax-increase-to-offset-debt-and-deficit",
                        "crime-punishment-level",
                        "military-hawkishness",
                        "non-military-interventions",
                        "affirmative-action",
                        "prayer-in-schools",
                        "market-solution-to-healthcare",
                        "moralistic-law",
                    ],
                    [
                        "government-intervention-weight",
                        "government-control-of-economy-weight",
                        "free-trade-weight",
                        "protectionism-weight",
                        "pro-choice-weight",
                        "tax-increase-to-offset-debt-and-deficit-weight",
                        "crime-punishment-level-weight",
                        "military-hawkishness-weight",
                        "non-military-interventions-weight",
                        "affirmative-action-weight",
                        "prayer-in-schools-weight",
                        "market-solution-to-healthcare-weight",
                        "moralistic-law-weight",
                    ],
                )
                if attr in self.attributes and weight_attr in self.attributes
            )

            if difference < min_difference:
                min_difference = difference
                chosen_candidate = candidate

        return chosen_candidate.unique_id  # Return the chosen candidate's unique ID



# Candidate Agent
class Candidate(Agent):
    def __init__(self, unique_id, model, attributes):
        super().__init__(unique_id, model)
        for attr, value in attributes.items():
            setattr(self, attr, value)
        self.attributes = attributes

In [21]:
# Main Model
class VotingModel(Model):
    def __init__(self, turtles_data, candidate_attributes, info_level_threshold=0):
        super().__init__()  # Call the parent class constructor
        self.schedule = RandomActivation(self)
        self.candidates = []

        # Add Candidate Agents
        for i, attrs in enumerate(candidate_attributes):
            candidate = Candidate(f"Candidate_{i}", self, attrs)
            self.schedule.add(candidate)
            self.candidates.append(candidate)

        # Add Voter Agents
        for _, row in turtles_data.iterrows():
            if row["information-level"] >= info_level_threshold:
                attributes = {
                    "government-intervention": row["government-intervention"],
                    "government-control-of-economy": row["government-control-of-economy"],
                    "free-trade": row["free-trade"],
                    "protectionism": row["protectionism"],
                    "pro-choice": row["pro-choice"],
                    "tax-increase-to-offset-debt-and-deficit": row["tax-increase-to-offset-debt-and-deficit"],
                    "crime-punishment-level": row["crime-punishment-level"],
                    "military-hawkishness": row["military-hawkishness"],
                    "non-military-interventions": row["non-military-interventions"],
                    "affirmative-action": row["affirmative-action"],
                    "prayer-in-schools": row["prayer-in-schools"],
                    "market-solution-to-healthcare": row["market-solution-to-healthcare"],
                    "moralistic-law": row["moralistic-law"],
                    "government-intervention-weight": row["government-intervention-weight"],
                    "government-control-of-economy-weight": row["government-control-of-economy-weight"],
                    "free-trade-weight": row["free-trade-weight"],
                    "protectionism-weight": row["protectionism-weight"],
                    "pro-choice-weight": row["pro-choice-weight"],
                    "tax-increase-to-offset-debt-and-deficit-weight": row["tax-increase-to-offset-debt-and-deficit-weight"],
                    "crime-punishment-level-weight": row["crime-punishment-level-weight"],
                    "military-hawkishness-weight": row["military-hawkishness-weight"],
                    "non-military-interventions-weight": row["non-military-interventions-weight"],
                    "affirmative-action-weight": row["affirmative-action-weight"],
                    "prayer-in-schools-weight": row["prayer-in-schools-weight"],
                    "market-solution-to-healthcare-weight": row["market-solution-to-healthcare-weight"],
                    "moralistic-law-weight": row["moralistic-law-weight"],
                }
                voter = Voter(row["who"], self, attributes)
                self.schedule.add(voter)

        # Data Collector
        self.datacollector = DataCollector(
            agent_reporters={
                "Vote": lambda agent: agent.evaluate_candidate() if isinstance(agent, Voter) else None,
                **{f"column_{i}": lambda agent, i=i: agent.attributes.get(f"column_{i}", None) for i in range(18, 44)}
    }
)


    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [22]:
if __name__ == "__main__":
    # Load your turtles_data DataFrame
    base_dir = "C:/Users/hugov/Github/Bachelor-project/Data/CSV"

    # Define the ranges for the file names
    range_1 = range(1, 21)
    range_2 = [500, 750]

    # Dictionary to store the data frames
    turtles_data = {}

    # Loop over the ranges and read the files dynamically
    for i in range_1:
        for j in range_2:
            # Construct the file name
            file_name = f"TURTLES_{i}_{j}.csv"
            file_path = os.path.join(base_dir, file_name)
        
            # Read the CSV file and store it in the dictionary
            if os.path.exists(file_path):
                data_name = f"Turtles{i}_{j}"
                turtles_data[data_name] = pd.read_csv(file_path)
            else:
                print(f"File not found: {file_path}")

    # Combine all DataFrames into one or use a specific DataFrame
    combined_turtles_data = pd.concat(turtles_data.values(), ignore_index=True)  # Option 1
    # specific_turtles_data = turtles_data.get("Turtles1_500")                  # Option 2

    # Define attributes for the candidates
    candidate_1_attributes = {
        "government-intervention": 10,
        "government-control-of-economy": 20,
        "free-trade": 30,
        "protectionism": 40,
        "pro-choice": 50,
        "tax-increase-to-offset-debt-and-deficit": 60,
        "crime-punishment-level": 70,
        "military-hawkishness": 80,
        "non-military-interventions": 90,
        "affirmative-action": 100,
        "prayer-in-schools": 110,
        "market-solution-to-healthcare": 120,
        "moralistic-law": 130
    }

    candidate_2_attributes = {
        "government-intervention": 15,
        "government-control-of-economy": 25,
        "free-trade": 35,
        "protectionism": 45,
        "pro-choice": 55,
        "tax-increase-to-offset-debt-and-deficit": 65,
        "crime-punishment-level": 75,
        "military-hawkishness": 85,
        "non-military-interventions": 95,
        "affirmative-action": 105,
        "prayer-in-schools": 115,
        "market-solution-to-healthcare": 125,
        "moralistic-law": 135
    }
    
    # Initialize the model with the desired DataFrame
    model = VotingModel(combined_turtles_data, [candidate_1_attributes, candidate_2_attributes])  # Use combined or specific

    # Run the model for 10 steps
    for i in range(10):
        model.step()

    # Save collected data to a CSV file
    agent_data = model.datacollector.get_agent_vars_dataframe()

    # Convert agent data to a format suitable for merging
    agent_data.reset_index(inplace=True)
    agent_data.rename(columns={"AgentID": "who"}, inplace=True)  # Ensure matching key for merging

    # Merge original turtles_data with the collected data
    final_data = pd.merge(combined_turtles_data, agent_data, on="who", how="left")

    # Save to CSV
    final_data.to_csv("Data/Voting_results/voting_results.csv", index=False)
    print("Voting results saved to voting_results.csv")



File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_1_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_2_500.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_2_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_3_500.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_3_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_4_500.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_4_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_5_500.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_5_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_6_500.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_6_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_7_500.csv
File not found: 